# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# File to Load (Remember to Change These)
city_data_load = "WeatherPy/WeatherPy.csv"
#student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
city_df = pd.read_csv(city_data_load)
#student_data = pd.read_csv(student_data_to_load)
city_df.dropna(inplace = True) 
city_df

,City,Country,Max Temp,Date,Cloudiness,Lat,Lng,Wind Speed,Humidity
0,Bredasdorp,ZA,50.00,1619892724,6,-34.5322,20.0403,2.30,93
1,Rikitea,PF,76.48,1619892685,88,-23.1203,-134.9692,11.03,79
2,Tiznit Province,MA,60.31,1619892724,98,29.5833,-9.5000,10.13,61
3,Nishihara,JP,71.60,1619892725,75,26.1842,127.7558,5.75,83
4,Flinders,AU,57.99,1619892725,74,-34.5833,150.8552,1.99,90
...,...,...,...,...,...,...,...,...,...
553,Riyadh,SA,89.60,1619892575,0,24.6877,46.7219,6.91,12
554,Husavik,IS,35.60,1619892867,75,66.0449,-17.3389,14.97,69
555,Bereda,ES,52.00,1619892867,95,43.2686,-7.5406,10.00,65
556,Bida,NG,95.83,1619892868,43,9.0833,6.0167,5.01,29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_df["Humidity"].astype(float)

In [12]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
city1 = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70)]
city2 = city1.loc[(city1["Wind Speed"] < 10)]
city3 = city2.loc[city2["Cloudiness"] == 0]
city3

,City,Country,Max Temp,Date,Cloudiness,Lat,Lng,Wind Speed,Humidity
8,Saint-Philippe,RE,75.20,1619892690,0,-21.3585,55.7679,6.91,78
70,Pisco,PE,71.60,1619892739,0,-13.7000,-76.2167,9.22,73
138,Sivota,GR,75.20,1619892754,0,39.4078,20.2452,4.61,60
203,Saint-Pierre,RE,75.20,1619892772,0,-21.3393,55.4781,6.91,78
270,Santa Isabel,BR,78.01,1619892789,0,-23.3156,-46.2214,5.75,49
487,Saint-Joseph,RE,75.20,1619892707,0,-21.3667,55.6167,6.91,78


In [14]:
#city4 = city3.loc[(city1["Wind Speed"] > 5)]
#city4

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = pd.DataFrame({})
hotel_df


hotel_df["Hotel Name"]= ""
hotel_df["City"] = city3["City"]
hotel_df["Country"]=city3["Country"]
hotel_df["Lat"] = ""
hotel_df["Lng"]=""


hotel_df

,Hotel Name,City,Country,Lat,Lng
8,NaN,Saint-Philippe,RE,,
70,NaN,Pisco,PE,,
138,NaN,Sivota,GR,,
203,NaN,Saint-Pierre,RE,,
270,NaN,Santa Isabel,BR,,
487,NaN,Saint-Joseph,RE,,


In [16]:

for index, row in city3.iterrows():
   

    lat = city3.loc[index,"Lat"]
    lng = city3.loc[index,"Lng"]
    target_radius = 5000
    target_type = "hotel"
    
    
    params = {
    "location": f"{lat},{lng}",
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
  
# base url
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
# run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
 
    geo = results[0]["geometry"]
    loc = geo["location"]
    lt = loc["lat"]
    ln = loc["lng"]
# Print the json (pretty printed)
    #print(json.dumps(results, indent=4, sort_keys=False))
    
    print(f"Retrieving Results for Index {index}")
    print(results[0]["name"])
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    hotel_df.loc[index, 'Lat'] = loc["lat"]
    hotel_df.loc[index, 'Lng'] = loc["lng"]
    
    print("======================")
    
        
        

Retrieving Results for Index 8
Saint-Philippe
Retrieving Results for Index 70
Pisco
Retrieving Results for Index 138
Syvota
Retrieving Results for Index 203
Saint Pierre
Retrieving Results for Index 270
Santa Isabel
Retrieving Results for Index 487
Saint-Joseph


In [17]:
hotel_df

,Hotel Name,City,Country,Lat,Lng
8,Saint-Philippe,Saint-Philippe,RE,-21.3581,55.7647
70,Pisco,Pisco,PE,-13.7135,-76.1842
138,Syvota,Sivota,GR,39.4075,20.2444
203,Saint Pierre,Saint-Pierre,RE,-21.3328,55.4718
270,Santa Isabel,Santa Isabel,BR,-23.3158,-46.2254
487,Saint-Joseph,Saint-Joseph,RE,-21.3804,55.6128


In [18]:
# geocoordinates
#target_coordinates = "-9.6658,-35.7353"
#print(target_coordinates)
#target_coordinates = city3["Lat"],city3["Lng"]
#target_radius = 3000
#target_type = "hotel"

# set up a parameters dictionary
#params = {
  #  "location": target_coordinates,
   # "radius": target_radius,
 #   "type": target_type,
 #   "key": g_key
#}

# base url
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
#response = requests.get(base_url, params=params).json()



In [19]:
# convert response to json
#places_data = response.json()
#results = response['results']
# Print the json (pretty printed)
#print(json.dumps(results, indent=4, sort_keys=False))

In [20]:
#print(results[0]["name"])
#print(results[0]["geometry"])
#g = results[0]["geometry"]
#a = g["location"]
#b = a["lat"]
#b

In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lng']]

# Display figure
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))